In [109]:
import tensorflow as tf
# Keras based Dependencies
import keras
from keras.models import Sequential,Model
from keras.datasets import mnist
from keras.optimizers import Adam
from keras.layers import Dense,Conv2D,MaxPool2D,Dropout,BatchNormalization,Activation,Flatten,merge
from keras.initializers import TruncatedNormal
from keras.losses import categorical_crossentropy
from keras.preprocessing import image
from keras.utils import to_categorical


import numpy as np
from scipy.io import loadmat
import h5py
import os
from IPython.display import Image

# To Grow GPU Usage dynamically when required
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

print("Keras Version : ",keras.__version__)
print("Tensorflow Version : ",tf.__version__)

Keras Version :  2.2.0
Tensorflow Version :  1.8.0


# Mnist Data Preparation :


In [142]:
(X_train,Y_train),(X_test,Y_test)=mnist.load_data()

In [143]:
X_train_fil=X_train[Y_train>=5]
Y_train_fil=Y_train[Y_train>=5]

X_test_fil=X_test[Y_test>=5]
Y_test_fil=Y_test[Y_test>=5]

# One Hot Encoding
num_classes = len(np.unique(Y_train_fil))
'''
Classes : 
6 -> 1 0 0 0
7 -> 0 1 0 0
8 -> 0 0 1 0
9 -> 0 0 0 1

'''
print(num_classes)
start_class= np.min(Y_train_fil)
Y_train_fil_cat= np.eye(num_classes)[Y_train_fil-start_class]
Y_test_fil_cat=np.eye(num_classes)[Y_test_fil-start_class]

5


In [144]:
print("Training Data", X_train_fil.shape,"Trainning Label",Y_train_fil_cat.shape)
print("Test Data", X_test_fil.shape,"Test Label",Y_test_fil_cat.shape)



Training Data (29404, 28, 28) Trainning Label (29404, 5)
Test Data (4861, 28, 28) Test Label (4861, 5)


# SVNH Data Preparation :

In [145]:
PathtoSVNH='..\\Datasets\\SVNH\\'
SVNH_tr=loadmat(PathtoSVNH+'train_32x32.mat')
SVNH_tst=loadmat(PathtoSVNH+'test_32x32.mat')
SVNH_train=SVNH_tr['X']
SVNH_train_label=SVNH_tr['y']
SVNH_test=SVNH_tst['X']
SVNH_test_label=SVNH_tst['y']

# Cleaning up the data into suitable standard format

SVNH_train=np.swapaxes(SVNH_train,0,3).swapaxes(2,3)
SVNH_test=np.swapaxes(SVNH_test,0,3).swapaxes(2,3)

SVNH_train_label = np.concatenate( SVNH_train_label, axis=0 )
SVNH_test_label = np.concatenate( SVNH_test_label, axis=0 )

#  '0' has label 10.
SVNH_train_label[SVNH_train_label==10]=0
SVNH_test_label[SVNH_test_label==10]=0

del SVNH_tr,SVNH_tst

In [147]:
SVNH_train_fil=SVNH_train[SVNH_train_label<5]
SVNH_train_label_fil=SVNH_train_label[SVNH_train_label<5]
SVNH_test_fil=SVNH_test[SVNH_test_label<5]
SVNH_test_label_fil=SVNH_test_label[SVNH_test_label<5]
# One Hot Encoding
'''
Classes : 
6 -> 1 0 0 0
7 -> 0 1 0 0
8 -> 0 0 1 0
9 -> 0 0 0 1

'''
print(num_classes)
start_class= np.min(SVNH_train_label_fil)

num_classes = len(np.unique(SVNH_train_label_fil))
SVNH_train_label_fil_cat= np.eye(num_classes)[SVNH_train_label_fil-start_class] # -1 to make the indexing count
SVNH_test_label_fil_cat=np.eye(num_classes)[SVNH_test_label_fil-start_class]

5


In [148]:
# Resize/Center Crop the image

SVNH_train_fil=SVNH_train_fil[:,2:30,2:30,:]
SVNH_test_fil=SVNH_test_fil[:,2:30,2:30,:]


In [149]:
print("Training Data", SVNH_train_fil.shape,"Trainning Label",SVNH_train_label_fil_cat.shape)
print("Test Data", SVNH_test_fil.shape,"Test Label",SVNH_test_label_fil_cat.shape)

Training Data (45349, 28, 28, 3) Trainning Label (45349, 5)
Test Data (16397, 28, 28, 3) Test Label (16397, 5)


# Use Lenet Architecture and Perform Domain Transfer Learning
# between SVNH 0-4 to MNIST 5-9



In [134]:
Image(url='https://world4jason.gitbooks.io/research-log/content/deepLearning/CNN/img/lenet.png')

In [150]:
Base_Model=Sequential()
Base_Model.add(Conv2D(batch_input_shape=(None,28,28,3),kernel_size=[5,5],filters=6))
Base_Model.add(MaxPool2D(pool_size=2, strides=2, padding='valid'))
Base_Model.add(Conv2D(kernel_size=[5,5],filters=16))
Base_Model.add(MaxPool2D(pool_size=2,strides=2,padding='valid'))
Base_Model.add(Flatten())
Base_Model.add(Dense(units=120,activation='sigmoid'))
Base_Model.add(Dense(units=84,activation='sigmoid'))
Base_Model.add(Dense(units=num_classes,activation='softmax'))


Base_Model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 24, 24, 6)         456       
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 12, 12, 6)         0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 8, 8, 16)          2416      
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 4, 4, 16)          0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 256)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 120)               30840     
_________________________________________________________________
dense_17 (Dense)             (None, 84)                10164     
__________

In [151]:
adam=keras.optimizers.Adam(lr=0.0001, beta_1=0.75, beta_2=0.99, epsilon=None, decay=0.0)


# Tensorboard Logging - Run only if required

In [152]:
tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)


# Model Trainning

In [153]:
Base_Model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])

In [154]:
Base_Model.fit(SVNH_train_fil, SVNH_train_label_fil_cat,batch_size=64,epochs=1,verbose=1,validation_data=(SVNH_test_fil, SVNH_test_label_fil_cat),callbacks=[tbCallBack])

Train on 45349 samples, validate on 16397 samples
Epoch 1/1
45349/45349 [==============================] - 41s 903us/step - loss: 1.4843 - acc: 0.3740 - val_loss: 1.3743 - val_acc: 0.4834


# Six different methods : 
* (i) Target only: the model is trained on D2 from scratch; 
* (ii) Fine-tune: the model is pretrained on D1 and fine-tuned on D2;
* (iii) Matching networks [70]: we first pretrain the model on D3, then use D2 as the support set in the matching networks; 
* (iv) Fine-tuned matching networks: same as baseline iii, except that for each k the model is fine-tuned on D2 with 5-way (k 􀀀 1)-shot learning: k 􀀀 1 examples in each class are randomly selected as the support set, and the last example in each class is used as the query set; 
* (v) Fine-tune + adversarial: in addition to baseline ii, the model is also trained on D1 and D3 with a domain adversarial loss; 
* (vi.) Full model: fine-tune the model with the proposed multi-layer domain adversarial loss.